In [3]:
import pandas as pd
import numpy as np
import pysam   
import collections
from cyvcf2 import VCF

df_breakpoints = pd.read_csv('../../../data/references/liu-data-reference/all_break_points.txt', sep='\t')
df_lookup = pd.read_csv('../../../data/references/liu-data-reference/liu_sample_lookup.tsv', sep='\t')
bam_files_path = '../../../data/liu-data/bam/'
vcf_files_path = '../../../data/liu-data/vcf/'

In [ ]:
'''
Grab reads from all_break_points.txt, look up their corresponding bam file in liu_sample_lookup.tsv
and write the reads into a text file
'''

missing = collections.Counter()

counter = 0

with open('out.txt', 'w') as f:
    for index, row in df_breakpoints.iterrows():
        
        individual = row[2]
        chromosome = row[3]
        left_bound = int(row[4])
        right_bound = int(row[5])
        
        counter += 1
        
        if counter % 100 == 0:
            print(missing)
        
        #try:
        run = df_lookup[df_lookup['individual'] == individual].iloc[0]['Run'] + '.dup.bam'
        #except:
        #    missing[run] += 1
         #   continue

        alignment_file = pysam.AlignmentFile(bam_files_path + run, 'rb')
        for alignment in alignment_file.fetch(chromosome, left_bound, right_bound):
            f.write(str(alignment)+'\n')
            
print(missing)

In [10]:
print(individual)

C3-9.1


In [16]:
'''
Create bam file using first region of all_break_points.txt. 10% of reads should be of that section and the
rest should be from other individuals without the recombination there

Recombination location:
C2-1.1 chromosome_1 left: 2031009 right: 2035598 length: 4589 SRR5243297.dup.bam
'''

'''
The file with the recombination is SRR5243297 and the other 9 are files without
the recombination between 2031009 and 2035598
'''

recomb_file = pysam.AlignmentFile(bam_files_path + 'SRR5243297' + '.dup.bam')
files = ['SRR5243270', 'SRR5243271', 'SRR5243272', 'SRR5243273', 
         'SRR5243274', 'SRR5243275', 'SRR5243276', 'SRR5243277', 'SRR5243278']
chromosome = 'chromosome_1'
left_bound = 2031009
right_bound = 2035598

f = pysam.AlignmentFile('recomb_mock.bam', 'wh', template=recomb_file)

counter = 0

for alignment in recomb_file:
    if counter >= 200:
        break
    f.write(alignment)
    counter += 1
    
for file in files:
    alignment_file = pysam.AlignmentFile(bam_files_path + file + '.dup.bam')

    counter = 0

    for alignment in alignment_file.fetch(chromosome, left_bound, right_bound):

        # write 200 lines from each file
        if counter >= 200:
            break

        f.write(alignment)
        counter += 1

In [10]:
'''
Using cyvcf, go through each of the breakpoints and check the number of SNPs per segment.
Return false if there is 1 or less SNPs. 
Otherwise, return the SNP locations in a list.
'''

def check_SNPs(f_name, chromosome, left_bound, right_bound):
    vcf_in = VCF(f_name)
    region = '{c}:{l}-{r}'.format(c=chromosome, l=left_bound, r=right_bound)
    records = [rec for rec in vcf_in(region)]
    
    if len(records) < 2:
        return False
    else:
        return records


In [48]:
'''
Given a bam file object, return the percentage of recombination in the bam files
'''

def recomb_percentage(bam_file_obj):
    
    recomb = 0
    total = 0
    
    for record in bam_file_obj:
        print(record.reference_name, record.query_start, record.query_end)
    

In [49]:
bam_file_obj = pysam.AlignmentFile('recomb_mock.bam', 'r')

recomb_percentage(bam_file_obj)

AttributeError: 'pysam.calignedsegment.AlignedSegment' object has no attribute 'query_start'

In [ ]:
with open('SNPs.txt', 'w') as f:
    for index, row in df_breakpoints.iterrows():

        individual = row[2]
        chromosome = row[3]
        left_bound = int(row[4])
        right_bound = int(row[5])

        #run = df_lookup[df_lookup['individual'] == individual].iloc[0]['Run'] + '.g.vcf.gz'
        
        for SNP in check_SNPs(vcf_files_path + 'H.vcf.gz', chromosome, left_bound, right_bound):
            print(SNP.POS, SNP.REF, SNP.ALT)
        
        break
        
        #f.write(str(check_SNPs(vcf_files_path + run, chromosome, left_bound, right_bound)) + '\n')
